In [14]:
import pandas as pd
import json

node_types_of_interest = ['Incoming Message', 'Bot Action']
edge_types_of_interest = ['leadsTo','uses','generates']




# import test_model.json
with open('test_model.json') as json_file:
    #load the json file
    json = json.load(json_file)

# dependency matrix
nodes = json['nodes']
edges = json['edges']

# create a dependency matrix as pandas dataframe 
df = pd.DataFrame(columns = nodes.keys(), index = nodes.keys())

for node_id in nodes.keys():
    if nodes[node_id]['type'] not in node_types_of_interest:
        continue
    df[node_id] = "#"
    df.loc[node_id] = "#" 

# fill the upper right triangle of the dependency matrix
#     -> means source_id depends on target_id
#     || means source_id depends on target_id and target_id depends on source_id
#     <- means target_id depends on source_id
#     # means no dependency

for edge in edges.values():
    if edge['type'] not in edge_types_of_interest:
            continue
    source_id = edge['source']
    target_id = edge['target']
    # get row and column index of the dependency matrix
    row_index = list(nodes.keys()).index(source_id)
    column_index = list(nodes.keys()).index(target_id)
    # fill the upper right triangle of the dependency matrix
    if row_index < column_index:
        if df.loc[source_id, target_id] == "#" and df.loc[target_id, source_id] == "#":
            df.loc[source_id, target_id] = "->"
        elif df.loc[target_id, source_id] == "->":
            df.loc[source_id, target_id] = "||"

# fill the lower left triangle of the dependency matrix
# will be symmetric to the upper right triangle
for row_index in range(len(nodes.keys())):
    for column_index in range(len(nodes.keys())):
        if row_index < column_index:
            if df.iloc[row_index, column_index] == "->":
                df.iloc[column_index, row_index] = "<-"
            else :
                df.iloc[column_index, row_index] = df.iloc[row_index, column_index]
    
df


,e8df33164e46ce87420e8af7,6c71b808ff99faedd986850e,a2328d74750c839062ebabb0,df08bdf6da892e42169acac2,3b995f08d360475619d5dead,b83a85b73e0b1ed74ed0091e,2cf567110bae35efe71a5cdf,117e0d66e833fb189e661891,2737c217b4ca167946a11531,a591cae876337f97bc28a95e,...,b3bb240b54166af91c94ce4b,0d6daa47e7c34171bba39dbd,9df99e13096da83d093358fb,f8f8142fd272fb35e933c7d4,7169854c636b80c0a1dd7639,df17b98b9ca57517d5eb134c,a913d82e77b1ec8edc5b566a,9b1dbda718ab29d02cf45d48,d94ff20cab73cf98782a9efc,ff9801f80454b4dbcedd4737
e8df33164e46ce87420e8af7,NaN,NaN,NaN,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
6c71b808ff99faedd986850e,NaN,NaN,NaN,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
a2328d74750c839062ebabb0,NaN,NaN,NaN,->,->,->,->,->,->,->,...,#,#,#,#,#,#,#,#,#,->
df08bdf6da892e42169acac2,#,#,<-,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
3b995f08d360475619d5dead,#,#,<-,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
b83a85b73e0b1ed74ed0091e,#,#,<-,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
2cf567110bae35efe71a5cdf,#,#,<-,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
117e0d66e833fb189e661891,#,#,<-,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
2737c217b4ca167946a11531,#,#,<-,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
a591cae876337f97bc28a95e,#,#,<-,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
